In [1]:
import time
import tensorflow as tf
import wandb
import pandas as pd
from prosit_t.wandb_agent.train_utils import get_proteometools_data
import seaborn as sns 
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import os
import matplotlib.pyplot as plt
from prosit_t.models import PrositTransformerV2
from dlomix.models import PrositIntensityPredictor

2023-09-20 09:01:39.954158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 09:01:40.759702: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-p

In [2]:
import plotly.io as pio
pio.renderers.default = "iframe"
import plotly.express as px

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
data_config = {
    "dataset": "proteometools",
    "data_source": {
        "train": "/cmnfs/proj/prosit/Transformer/first_pool_train.parquet",
        "val": "/cmnfs/proj/prosit/Transformer/first_pool_test.parquet",
    },
    "fragmentation": "HCD",
    "batch_size": 1024,
    "seq_length": 30,
}

In [5]:
project_name = "transforming-prosit-first-pool"
run = wandb.init(project=project_name)

wandb: Currently logged in as: mamisashvili-lizi (prosit-compms). Use `wandb login --relogin` to force relogin


In [6]:
_, val_data = get_proteometools_data(data_config)

2023-09-20 09:02:38.883722: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43640 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6


In [7]:
def compute_losses(model, val_data, num_batches, batch_size):
    losses = []
    for sample in val_data.take(num_batches):
        x, y = sample
        predictions = model.predict(x, batch_size=batch_size)
        loss = masked_spectral_distance(y, predictions)
        losses.extend(loss.numpy())
    return losses

In [38]:
artifact_path = "prosit-compms/transforming-prosit-first-pool/model-lunar-yogurt-85:v108"

In [39]:
artifact = run.use_artifact(artifact_path)
model_dir = artifact.download()
transformer = PrositTransformerV2(seq_length=30,embedding_output_dim=64,num_heads=8,
        num_transformers=6, dense_dim_factor=4)
transformer.load_weights(model_dir)

wandb:   5 of 5 files downloaded.  
2023-09-20 09:24:42.058062: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./artifacts/model-lunar-yogurt-85:v108: FAILED_PRECONDITION: artifacts/model-lunar-yogurt-85:v108; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [40]:
num_batches = 600
batch_size = 1024

In [41]:
start_time = time.time()
transformer_loss = compute_losses(transformer, val_data, num_batches, batch_size)
end_time = time.time()
end_time - start_time

1/1 [==============================] - 0s 14ms/step


2023-09-20 09:24:46.545895: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11080" } environment { key: "cudnn" value: "8600" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45760577536 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 14ms/step


43.8992178440094

In [12]:
baseline_path = "prosit-compms/transforming-prosit-first-pool/model-classic-star-15:v45"
baseline_artifact = run.use_artifact(baseline_path)
baseline_dir = baseline_artifact.download()

wandb:   4 of 4 files downloaded.  


In [13]:
baseline = PrositIntensityPredictor(seq_length=30,embedding_output_dim=16,
        recurrent_layers_sizes=(256, 512))
baseline.load_weights(baseline_dir)

2023-09-20 09:03:55.678357: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./artifacts/model-classic-star-15:v45: FAILED_PRECONDITION: artifacts/model-classic-star-15:v45; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


In [14]:
start_time = time.time()
baseline_loss = compute_losses(baseline, val_data, num_batches, batch_size)
end_time = time.time()
end_time - start_time

1/1 [==============================] - 0s 26ms/step


2023-09-20 09:03:58.733699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 13ms/step


43.43578028678894

In [42]:
df_transformer = pd.DataFrame({"loss": transformer_loss, "model": "transformer"})
df_baseline = pd.DataFrame({"loss": baseline_loss, "model": "baseline"})

In [43]:
loss_df = pd.concat([df_transformer,df_baseline])

In [44]:
fig = px.violin(loss_df, y = 'loss', color = 'model', violinmode = 'overlay', box=True, points = False)

fig.show()

In [45]:
# Specify the file format and file name
file_format = 'png'
file_name = 'plotly_image'
file_path = f'{file_name}.{file_format}'

In [46]:
# Save the image
pio.write_image(fig, file_path, format=file_format)

In [36]:
px.histogram(loss_df[(loss_df.model=="transformer") & (loss_df.loss > 0.5)].loss,nbins=50)

In [37]:
px.histogram(loss_df[(loss_df.model=="baseline") & (loss_df.loss > 0.5)].loss,color_discrete_sequence=['indianred'])